# Chapter 3 DataFrame
不可变分布式数据集，类似于关系数据库中的表

## Python  与 RDD 之间的通信

Whenever a PySpark program is executed using RDDs, there is a potentially large overhead to execute the job.
In the PySpark driver, the Spark Context uses Py4j to launch a JVM using the JavaSparkContext. Any RDD transformations are initially mapped to PythonRDD objects in Java.
While this approach allows PySpark to distribute the processing of the data to multiple Python subprocesses on multiple workers, as you can see, there is a lot of context switching and communications overhead between Python and the JVM.

## Catalyst Optimizer refresh

![image.png](attachment:assets/image.png)

This diagram looks similar to the logical/physical planner and cost model/cost-based optimization of a relational database management system (RDBMS):

The significance of this is that, as opposed to immediately processing the query, the Spark engine's Catalyst Optimizer compiles and optimizes a logical plan and has a cost optimizer that determines the most efficient physical plan generated.

## Speeding up PySpark with DataFrames

The significance of DataFrames and the Catalyst Optimizer (and Project Tungsten) is the increase in performance of PySpark queries when compared to non-optimized RDD queries.
Prior to the introduction of DataFrames, Python query speeds were often twice as slow as the same Scala queries using RDD. Typically, this slowdown in query performance was due to the communications overhead between Python and the JVM。

## 创建 DataFrame

First, instead of accessing the file system, we will create a DataFrame by generating the data. In this case, we'll first create the stringJSONRDD RDD and then convert it into a DataFrame. This code snippet creates an RDD comprised of swimmers (their ID, name, age, and eye color) in JSON format.

### Generating our own JSON data


In [2]:
from pyspark import SparkContext , SparkConf
conf = SparkConf().setAppName('chapter3')
sc = SparkContext(conf=conf)
sc

<SparkContext master=local[*] appName=chapter3>

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Chapter3") \
    .getOrCreate()

In [3]:
# Generate our own JSON data 
#   This way we don't have to access the file system yet.
stringJSONRDD = sc.parallelize((""" 
  { "id": "123",
    "name": "Katie",
    "age": 19,
    "eyeColor": "brown"
  }""",
   """{
    "id": "234",
    "name": "Michael",
    "age": 22,
    "eyeColor": "green"
  }""", 
  """{
    "id": "345",
    "name": "Simone",
    "age": 23,
    "eyeColor": "blue"
  }""")
)

Now that we have created the RDD, we will convert this into a DataFrame by using the SparkSession `read.json` method (that is, spark.read.json(...)). We will also create a temporary table by using the .createOrReplaceTempView method.

### Creating a DataFrame

In [8]:
swimmersJSON = spark.read.json(stringJSONRDD)
swimmersJSON

DataFrame[age: bigint, eyeColor: string, id: string, name: string]

### Creating a temporary table

In [9]:
swimmersJSON.createOrReplaceTempView("swimmersJSON")

As noted in the previous chapters, many RDD operations are transformations, which are not executed until an action operation is executed. For example, in the preceding code snippet, the sc.parallelize is a transformation that is executed when converting from an RDD to a DataFrame by using spark.read.json.

Note that creating the temporary table is a DataFrame transformation and not executed until a DataFrame action is executed

## Simple DataFrame queries

### DataFrame API query

In [10]:
swimmersJSON.show()

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 19|   brown|123|  Katie|
| 22|   green|234|Michael|
| 23|    blue|345| Simone|
+---+--------+---+-------+



### SQL query

In [11]:
spark.sql("select * from swimmersJSON").collect()

[Row(age=19, eyeColor='brown', id='123', name='Katie'),
 Row(age=22, eyeColor='green', id='234', name='Michael'),
 Row(age=23, eyeColor='blue', id='345', name='Simone')]

Note that you can use either the `collect()` or `show()` method for both DataFrames and SQL queries. Just make sure that if you use `.collect()`, this is for a small DataFrame, since it will return all of the rows in the DataFrame and move them back from the executors to the driver. You can instead use `take(<n>)` or `show(<n>)`, which allow you to limit the number of rows returned by specifying `<n>`

## RDD 的交互

There are two different methods for converting existing RDDs to DataFrames (or Datasets[T]): inferring the schema using reflection, or programmatically specifying the schema. The former allows you to write more concise code (when your Spark application already knows the schema), while the latter allows you to construct DataFrames when the columns and their data types are only revealed at run time. Note, reflection is in reference to schema reflection as opposed to Python reflection.

### Inferring the schema using reflection

In the process of building the DataFrame and running the queries, we skipped over the fact that the schema for this DataFrame was automatically defined. Initially, row objects are constructed by passing a list of key/value pairs as **kwargs to the row class. Then, Spark SQL converts this RDD of row objects into a DataFrame, where the keys are the columns and the data types are inferred by sampling the data.

In [12]:
swimmersJSON.printSchema()

root
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)



### Programmatically specifying the schema

In this case, let's programmatically specify the schema by bringing in Spark SQL data types (pyspark.sql.types) and generate some .csv data for this example:

In [13]:
from pyspark.sql.types import *

# Generate our own CSV data 
stringCSVRDD = sc.parallelize([(123, 'Katie', 19, 'brown'), (234, 'Michael', 22, 'green'), (345, 'Simone', 23, 'blue')])

# The schema is encoded in a string, using StructType we define the schema using various pyspark.sql.types
schemaString = "id name age eyeColor"
schema = StructType([
    StructField("id", LongType(), True),    
    StructField("name", StringType(), True),
    StructField("age", LongType(), True),
    StructField("eyeColor", StringType(), True)
])

# Apply the schema to the RDD and Create DataFrame
swimmers = spark.createDataFrame(stringCSVRDD, schema)

# Creates a temporary view using the DataFrame
swimmers.createOrReplaceTempView("swimmers")

Note, the StructField class is broken down in terms of:
- name: The name of this field
- dataType: The data type of this field
- nullable: Indicates whether values of this field can be null

In [14]:
swimmers.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)



## Querying with the DataFrame API

As noted in the previous section, you can start off by using `collect()`, `show()`, or `take()` to view the data within your `DataFrame` (with the last two including the option to limit the number of returned rows).

### Number of rows

In [15]:
swimmers.count()

3

### Running filter statements

To run a filter statement, you can use the `filter` clause; in the following code snippet, we are using the `select` clause to specify the columns to be returned as well:

In [16]:
# Get the id, age where age = 22
swimmers.select("id", "age").filter("age = 22").show()
# Another way to write the above query is below
swimmers.select(swimmers.id, swimmers.age).filter(swimmers.age == 22). show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



If we only want to get back the name of the swimmers who have an eye color that begins with the letter b, we can use a SQL-like syntax, `like`, as shown in the following code:

In [18]:
# Get the name, eyeColor where eyeColor like 'b%'
swimmers.select("name", "eyeColor").filter("eyeColor like 'b%'"). show()

+------+--------+
|  name|eyeColor|
+------+--------+
| Katie|   brown|
|Simone|    blue|
+------+--------+



## Querying with SQL

Let's run the same queries, except this time, we will do so using SQL queries against the same `DataFrame`. Recall that this DataFrame is accessible because we executed the `.createOrReplaceTempView` method for `swimmers`.

### Number of Rows

In [19]:
spark.sql("select count(1) from swimmers").show()

+--------+
|count(1)|
+--------+
|       3|
+--------+



### Running filter statements using the `where` Clauses

In [20]:
# Get the id, age where age = 22 in SQL

spark.sql("select id, age from swimmers where age = 22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+

